# Model training

## Installs

In [1]:
!pip install dm-sonnet folium==0.2.1 imgaug==0.2.6 Jinja2==2.11.3
#!pip install einops
#!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 254 kB 5.1 MB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 
     |████████████████████████████████| 631 kB 78.9 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=416c74272a01b4e97bfc67a95ec430b24e3a96cab990eaf9e284eeaaa7d6f716
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654017 sha256=c09e2b74bd5038307dde53cd093911859fc22f89474356ce9b90ce40e0af0668
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built folium imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.4.0
    Uninstalling imgaug-0.4.0:
      Successfully uninstalled imgaug-0.4.0
  Attempting uninstall: folium

## Google cloud authentication

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
from google.colab import auth
auth.authenticate_user()

## Imports

In [53]:
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.client import device_lib
from tensorflow.keras import Input
from tensorflow.keras.initializers import Identity, glorot_uniform, Zeros, he_normal
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Conv1D, BatchNormalization, Add , Activation , Lambda , Dropout, LayerNormalization, Input, Layer
from tensorflow.keras.regularizers import l1_l2, l2, l1
from tensorflow.keras.backend import conv1d
from tensorflow.keras.models import Model
from tensorflow.python.keras.utils import conv_utils
from tensorflow.keras.activations import gelu, relu, swish
from tensorflow.keras import backend as K
import tensorflow as tf
import networkx as nx
import scipy
from sklearn.preprocessing import LabelEncoder
import logging
import numpy as np
import pandas as pd
import h5py
import os
import datetime
import csv

In [5]:
import math
#from einops.layers.tensorflow import Rearrange, Reduce

## Setting up TPU access

In [6]:
#Get a handle to the attached TPU. On GCP it will be the CloudTPU itself
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#Connect to the TPU handle and initialise it
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.1.0.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.1.0.146:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [7]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Model code

In [9]:
wd = '/content/drive/MyDrive/Colab Notebooks/DREAM/Final/'

In [10]:

#import sys
#sys.path.insert(1, wd)

#from helpers import *

### Enformer attention module

In [11]:
## Copy-pasted from here: https://github.com/deepmind/deepmind-research/blob/master/enformer/attention_module.py
from typing import Any, Dict, List, Optional

import numpy as np
import sonnet as snt
import tensorflow as tf

class MultiheadAttention(snt.Module):
  """Multi-head attention."""

  def __init__(self,
               value_size: int,
               key_size: int,
               num_heads: int,
               scaling: bool = True,
               attention_dropout_rate: float = 0.1,
               relative_positions: bool = False,
               relative_position_symmetric: bool = False,
               relative_position_functions: Optional[List[str]] = None,
               num_relative_position_features: Optional[int] = None,
               positional_dropout_rate: float = 0.1,
               zero_initialize: bool = True,
               initializer: Optional[snt.initializers.Initializer] = None,
               name: str = None):
    """Creates a MultiheadAttention module.
    Args:
      value_size: The size of each value embedding per head.
      key_size: The size of each key and query embedding per head.
      num_heads: The number of independent queries per timestep.
      scaling: Whether to scale the attention logits.
      attention_dropout_rate: Dropout rate for attention logits.
      relative_positions: Whether to use TransformerXL style relative attention.
      relative_position_symmetric: If True, the symmetric version of basis
        functions will be used. If False, a symmetric and asymmetric versions
        will be use.
      relative_position_functions: List of function names used for relative
        positional biases.
      num_relative_position_features: Number of relative positional features
        to compute. If None, `value_size * num_heads` is used.
      positional_dropout_rate: Dropout rate for the positional encodings if
        relative positions are used.
      zero_initialize: if True, the final linear layer will be 0 initialized.
      initializer: Initializer for the projection layers. If unspecified,
        VarianceScaling is used with scale = 2.0.
      name: Name of module.
    """
    super().__init__(name=name)
    self._value_size = value_size
    self._key_size = key_size
    self._num_heads = num_heads
    self._attention_dropout_rate = attention_dropout_rate
    self._scaling = scaling
    self._relative_positions = relative_positions
    self._relative_position_symmetric = relative_position_symmetric
    self._relative_position_functions = relative_position_functions
    if num_relative_position_features is None:
      # num_relative_position_features needs to be divisible by the number of
      # relative positional functions *2 (for symmetric & asymmetric version).
      divisible_by = 2 * len(self._relative_position_functions)
      self._num_relative_position_features = (
          (self._value_size // divisible_by) * divisible_by)
    else:
      self._num_relative_position_features = num_relative_position_features
    self._positional_dropout_rate = positional_dropout_rate

    self._initializer = initializer
    if self._initializer is None:
      self._initializer = snt.initializers.VarianceScaling(scale=2.0)

    key_proj_size = self._key_size * self._num_heads
    embedding_size = self._value_size * self._num_heads

    self._q_layer = snt.Linear(
        key_proj_size,
        name='q_layer',
        with_bias=False,
        w_init=self._initializer)
    self._k_layer = snt.Linear(
        key_proj_size,
        name='k_layer',
        with_bias=False,
        w_init=self._initializer)
    self._v_layer = snt.Linear(
        embedding_size,
        name='v_layer',
        with_bias=False,
        w_init=self._initializer)
    w_init = snt.initializers.Zeros() if zero_initialize else self._initializer
    self._embedding_layer = snt.Linear(
        embedding_size,
        name='embedding_layer',
        w_init=w_init)

    # Create additional layers if using relative positions.
    if self._relative_positions:
      self._r_k_layer = snt.Linear(
          key_proj_size,
          name='r_k_layer',
          with_bias=False,
          w_init=self._initializer)
      self._r_w_bias = tf.Variable(
          self._initializer([1, self._num_heads, 1, self._key_size],
                            dtype=tf.float32),
          name='r_w_bias')
      self._r_r_bias = tf.Variable(
          self._initializer([1, self._num_heads, 1, self._key_size],
                            dtype=tf.float32),
          name='r_r_bias')

  def _multihead_output(self, linear, inputs):
    """Applies a standard linear to inputs and returns multihead output."""

    output = snt.BatchApply(linear)(inputs)  # [B, T, H * KV]
    num_kv_channels = output.shape[-1] // self._num_heads
    # Split H * Channels into separate axes.
    output = snt.reshape(output,
                         output_shape=[-1, self._num_heads, num_kv_channels])
    # [B, T, H, KV] -> [B, H, T, KV]
    return tf.transpose(output, [0, 2, 1, 3])

  def __call__(self,
               inputs,
               is_training=False):
    # Initialise the projection layers.
    embedding_size = self._value_size * self._num_heads
    seq_len = inputs.shape[1]

    # Compute q, k and v as multi-headed projections of the inputs.
    q = self._multihead_output(self._q_layer, inputs)  # [B, H, T, K]
    k = self._multihead_output(self._k_layer, inputs)  # [B, H, T, K]
    v = self._multihead_output(self._v_layer, inputs)  # [B, H, T, V]

    # Scale the query by the square-root of key size.
    if self._scaling:
      q *= self._key_size**-0.5

    if self._relative_positions:
      # For relative positions, we project positions to form relative keys.
      distances = tf.range(-seq_len + 1, seq_len, dtype=tf.float32)[tf.newaxis]
      positional_encodings = positional_features_all(
          positions=distances,
          feature_size=self._num_relative_position_features,
          seq_length=seq_len,
          feature_functions=self._relative_position_functions,
          symmetric=self._relative_position_symmetric)
      # [1, 2T-1, Cr]

      if is_training:
        positional_encodings = tf.nn.dropout(
            positional_encodings, rate=self._positional_dropout_rate)

      # [1, H, 2T-1, K]
      r_k = self._multihead_output(self._r_k_layer, positional_encodings)

      # Add shifted relative logits to content logits.
      # [B, H, T', T]
      content_logits = tf.matmul(q + self._r_w_bias, k, transpose_b=True)
      # [B, H, T', 2T-1]
      relative_logits = tf.matmul(
          q + self._r_r_bias, r_k, transpose_b=True)
      #  [B, H, T', T]
      relative_logits = relative_shift(relative_logits)
      logits = content_logits + relative_logits
    else:
      # [B, H, T', T]
      logits = tf.matmul(q, k, transpose_b=True)

    weights = tf.nn.softmax(logits)

    # Dropout on the attention weights.
    if is_training:
      weights = tf.nn.dropout(weights, rate=self._attention_dropout_rate)

    # Transpose and reshape the output.
    output = tf.matmul(weights, v)  # [B, H, T', V]
    output_transpose = tf.transpose(output, [0, 2, 1, 3])  # [B, T', H, V]

    # Final linear layer.
    attended_inputs = snt.reshape(
        output_transpose, output_shape=[embedding_size], preserve_dims=2)
    output = self._embedding_layer(attended_inputs)

    return output


def relative_shift(x):
  """Shift the relative logits like in TransformerXL."""
  # We prepend zeros on the final timescale dimension.
  to_pad = tf.zeros_like(x[..., :1])
  x = tf.concat([to_pad, x], -1)
  _, num_heads, t1, t2 = x.shape
  x = tf.reshape(x, [-1, num_heads, t2, t1])
  x = tf.slice(x, [0, 0, 1, 0], [-1, -1, -1, -1])
  x = tf.reshape(x, [-1, num_heads, t1, t2 - 1])
  x = tf.slice(x, [0, 0, 0, 0], [-1, -1, -1, (t2 + 1) // 2])
  return x


# Available feature functions:
def get_positional_feature_function(name):
  """Returns positional feature functions."""
  available = {
      'positional_features_exponential': positional_features_exponential,
      'positional_features_central_mask': positional_features_central_mask,
      'positional_features_gamma': positional_features_gamma,
      'positional_features_cosine': positional_features_cosine,
      'positional_features_linear_masks': positional_features_linear_masks,
      'positional_features_sin_cos': positional_features_sin_cos,
  }
  if name not in available:
    raise ValueError(f'Function {name} not available in {available.keys()}')
  return available[name]


def positional_features_all(positions: tf.Tensor,
                            feature_size: int,
                            seq_length: Optional[int] = None,
                            bin_size: Optional[int] = None,
                            feature_functions: Optional[List[str]] = None,
                            symmetric=False):
  """Compute relative positional encodings/features.
  Each positional feature function will compute/provide the same fraction of
  features, making up the total of feature_size.
  Args:
    positions: Tensor of relative positions of arbitrary shape.
    feature_size: Total number of basis functions.
    seq_length: Sequence length denoting the characteristic length that
      the individual positional features can use. This is required since the
      parametrization of the input features should be independent of `positions`
      while it could still require to use the total number of features.
    bin_size: Bin sized used to partition the sequence. This can be used to
      compute features on the absolute scale relative to the genome.
    feature_functions: List of different feature functions to use. Each function
      will take as argument: positions, sequence length and number of features
      to compute.
    symmetric: If True, the resulting features will be symmetric across the
      relative position of 0 (i.e. only absolute value of positions will
      matter). If false, then both the symmetric and asymmetric version
      (symmetric multiplied by sign(positions)) of the features will be used.
  Returns:
    Tensor of shape: `positions.shape + (feature_size,)`.
  """
  if feature_functions is None:
    feature_functions = ['positional_features_exponential',
                         'positional_features_central_mask',
                         'positional_features_gamma']
  num_components = len(feature_functions)  # 1 per each basis function
  if not symmetric:
    num_components = 2 * num_components

  # For now, we do not allow odd sized embeddings.
  if feature_size % num_components != 0:
    raise ValueError(
        f'feature_size has to be divisible by {num_components}')

  feature_functions = [get_positional_feature_function(f)
                       for f in feature_functions]
  num_basis_per_class = feature_size // num_components
  embeddings = tf.concat([f(tf.abs(positions), num_basis_per_class,
                            seq_length, bin_size)
                          for f in feature_functions],
                         axis=-1)
  if not symmetric:
    embeddings = tf.concat([embeddings,
                            tf.sign(positions)[..., tf.newaxis] * embeddings],
                           axis=-1)
  tf.TensorShape(embeddings.shape).assert_is_compatible_with(
      positions.shape + [feature_size])
  return embeddings


def _prepend_dims(x, num_dims):
  return tf.reshape(x, shape=[1] * num_dims + x.shape)


def positional_features_exponential(positions: tf.Tensor,
                                    feature_size: int,
                                    seq_length: Optional[int] = None,
                                    bin_size: Optional[int] = None,
                                    min_half_life: Optional[float] = 3.0):
  """Create exponentially decaying positional weights.
  Args:
    positions: Position tensor (arbitrary shape).
    feature_size: Number of basis functions to use.
    seq_length: Sequence length.
    bin_size: (unused). See `positional_features_all`.
    min_half_life: Smallest exponential half life in the grid of half lives.
  Returns:
    A Tensor with shape [2 * seq_length - 1, feature_size].
  """
  del bin_size  # Unused.
  if seq_length is None:
    seq_length = tf.reduce_max(tf.abs(positions)) + 1
  # Grid of half lifes from [3, seq_length / 2] with feature_size
  # distributed on the log scale.
  seq_length = tf.cast(seq_length, dtype=tf.float32)
  max_range = tf.math.log(seq_length) / tf.math.log(2.0)
  half_life = tf.pow(2.0, tf.linspace(min_half_life, max_range, feature_size))
  half_life = _prepend_dims(half_life, positions.shape.rank)
  positions = tf.abs(positions)
  outputs = tf.exp(-tf.math.log(2.0) / half_life * positions[..., tf.newaxis])
  tf.TensorShape(outputs.shape).assert_is_compatible_with(
      positions.shape + [feature_size])
  return outputs


def positional_features_central_mask(positions: tf.Tensor,
                                     feature_size: int,
                                     seq_length: Optional[int] = None,
                                     bin_size: Optional[int] = None):
  """Positional features using a central mask (allow only central features)."""
  del seq_length  # Unused.
  del bin_size  # Unused.
  center_widths = tf.pow(2.0, tf.range(1, feature_size + 1, dtype=tf.float32))
  center_widths = center_widths - 1
  center_widths = _prepend_dims(center_widths, positions.shape.rank)
  outputs = tf.cast(center_widths > tf.abs(positions)[..., tf.newaxis],
                    tf.float32)
  tf.TensorShape(outputs.shape).assert_is_compatible_with(
      positions.shape + [feature_size])
  return outputs


def gamma_pdf(x, concentration, rate):
  """Gamma probability distribution function: p(x|concentration, rate)."""
  log_unnormalized_prob = tf.math.xlogy(concentration - 1., x) - rate * x
  log_normalization = (tf.math.lgamma(concentration) -
                       concentration * tf.math.log(rate))
  return tf.exp(log_unnormalized_prob - log_normalization)


def positional_features_gamma(positions: tf.Tensor,
                              feature_size: int,
                              seq_length: Optional[int] = None,
                              bin_size: Optional[int] = None,
                              stddev=None,
                              start_mean=None):
  """Positional features computed using the gamma distributions."""
  del bin_size  # Unused.
  if seq_length is None:
    seq_length = tf.reduce_max(tf.abs(positions)) + 1
  if stddev is None:
    stddev = seq_length / (2 * feature_size)
  if start_mean is None:
    start_mean = seq_length / feature_size
  mean = tf.linspace(start_mean, seq_length, num=feature_size)
  mean = _prepend_dims(mean, positions.shape.rank)
  concentration = (mean / stddev)**2
  rate = mean / stddev**2
  probabilities = gamma_pdf(
      tf.abs(tf.cast(positions, dtype=tf.float32))[..., tf.newaxis],
      concentration, rate)
  probabilities += 1e-8  # To ensure numerical stability.
  outputs = probabilities / tf.reduce_max(probabilities,
                                          axis=1, keepdims=True)
  tf.TensorShape(outputs.shape).assert_is_compatible_with(
      positions.shape + [feature_size])
  return outputs


def positional_features_cosine(positions: tf.Tensor,
                               feature_size: int,
                               seq_length: Optional[int] = None,
                               bin_size: Optional[int] = None):
  """Cosine positional features."""
  del bin_size  # Unused.
  del seq_length  # Unused.
  periodicity = 1.25 * tf.pow(2.0, tf.range(0, feature_size, dtype=tf.float32))
  periodicity = _prepend_dims(periodicity, positions.shape.rank)

  outputs = tf.math.cos(2 * np.pi * positions[..., tf.newaxis] / periodicity)
  tf.TensorShape(outputs.shape).assert_is_compatible_with(
      positions.shape + [feature_size])
  return outputs


def positional_features_linear_masks(positions: tf.Tensor,
                                     feature_size: int,
                                     seq_length: Optional[int] = None,
                                     bin_size: Optional[int] = None):
  """Exponentially increasing point focuses."""
  del bin_size  # Unused.
  del seq_length  # Unused.
  distances = tf.range(0, feature_size, dtype=tf.float32)
  distances = _prepend_dims(distances, positions.shape.rank)
  outputs = tf.cast(distances == tf.abs(positions[..., tf.newaxis]),
                    dtype=tf.float32)

  tf.TensorShape(outputs.shape).assert_is_compatible_with(
      positions.shape + [feature_size])
  return outputs


def positional_features_sin_cos(positions: tf.Tensor,
                                feature_size: int,
                                seq_length: Optional[int] = None,
                                bin_size: Optional[int] = None,
                                max_time=10000.0):
  """Sine/cosine positional encodings."""
  del bin_size  # Unused.
  del seq_length  # Unused.
  if feature_size % 2 != 0:
    raise ValueError('feature_size needs to be divisible by 2.')
  i = tf.range(0, feature_size, 2, dtype=tf.float32)
  i = _prepend_dims(i, positions.shape.rank)

  # Concat sines and cosines and return.
  outputs = tf.concat([
      tf.sin(positions[..., tf.newaxis] / max_time**(i / feature_size)),
      tf.cos(positions[..., tf.newaxis] / max_time**(i / feature_size))], -1)

  tf.TensorShape(outputs.shape).assert_is_compatible_with(
      positions.shape + [feature_size])
  return outputs

## Loading data

In [12]:
with h5py.File(wd + 'train_onehot_sequences_bool_half.h5', 'r') as hf:
    train_sequences = hf['onehot_sequences_bool'][:] 

with h5py.File(wd + 'train_expression_half.h5', 'r') as hf:
    train_expression = hf['expression'][:] 

In [13]:
# Define train-validation sizes 
Total_sequences = len(train_expression) 
train_split = 0.98
validation_split = 0.02
num_train = int(Total_sequences * train_split)
num_validation = int(Total_sequences * validation_split)

In [14]:
print('Training sequences : {}'.format(num_train))
print('Validation sequences : {}'.format(num_validation))

Training sequences : 4703150
Validation sequences : 95982


In [15]:
# Make TF dataset and do shuffling and splits
full_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_expression))

full_dataset = full_dataset.shuffle(buffer_size = 1024)
train_dataset = full_dataset.take(num_train)
validation_dataset = full_dataset.skip(num_train)

In [16]:
# Define train steps 
N_train_batches = len(train_dataset) // 128
N_val_batches = len(validation_dataset) // 128

## Model

### Logging and reporting

In [23]:
# Function to compute pearson correlation
def correlation_coefficient(y_true, y_pred): # Copied from https://github.com/1edv/evolution/blob/master/manuscript_code/model/tpu_model/rr_aux.py
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

def r_square(y_true, y_pred): # Copied from https://github.com/1edv/evolution/blob/master/manuscript_code/model/tpu_model/rr_aux.py
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [18]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
history = LossHistory()

csvlogger = keras.callbacks.CSVLogger(wd+"loss_history_Final_model.tsv", separator='\t', append=False)


model_path = wd + "Final_model.h5"
checkpoint = keras.callbacks.ModelCheckpoint(model_path, monitor='val_correlation_coefficient', 
                                             verbose=2, save_best_only=True, mode='max', save_weights_only = True)
early_stop = keras.callbacks.EarlyStopping(monitor='val_correlation_coefficient', patience=20, mode='max') 
callbacks_list = [checkpoint, early_stop, history , csvlogger ]


### Model parameters and architecture

In [43]:
batch_size= 1024 
input_shape = (batch_size // 8, 110, 4)

In [44]:
# Training params
epochs = 10
batch_size= int(1024*1) 
lr=0.0001
loss = 'mean_squared_error'
device_type ='tpu'

# CNN params
conv_size0 = 15
conv_size1 = 20
conv_size2 = 25
conv_size3 = 30
reduce_hidden = 96
cnn_activation = 'swish'
cnn_initializer = 'he_normal'

# LSTM params
lstm_activation = 'swish'
lstm_initializer = 'he_normal'
lstm_units = 4

# Transformer params
num_heads = 1
transformer_att_dropout = 0.0
transformer_pos_dropout = 0.0
transformer_rel_pos = True
transformer_pos_fncs = ['positional_features_exponential','positional_features_central_mask','positional_features_gamma']

# Output param
output_initializer = 'he_normal'

# Regularization
l1_weight= 0
l2_weight= 0.001

In [45]:
## Current best
def create_model():
  input_layer = Input(batch_shape=(int(batch_size / 8),input_shape[1],input_shape[2]))  #Batches are split into 8 "blocks" on the tpu

  # Convolutional block
  x_0 = Conv1D(reduce_hidden * 2, kernel_size = conv_size0, padding='same' , kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(input_layer) 
  x_0 = BatchNormalization()(x_0)
  x_0 = Activation(cnn_activation)(x_0)
  x_0 = Conv1D(reduce_hidden, kernel_size = 1, padding='same' , kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(x_0) 
  x_0 = BatchNormalization()(x_0)
  x_0 = Activation(cnn_activation)(x_0)

  x_1 = Conv1D(reduce_hidden * 2, kernel_size = conv_size1, padding='same' , kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(input_layer) 
  x_1 = BatchNormalization()(x_1)
  x_1 = Activation(cnn_activation)(x_1)
  x_1 = Conv1D(reduce_hidden, kernel_size = 1, padding='same' , kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(x_1) 
  x_1 = BatchNormalization()(x_1)
  x_1 = Activation(cnn_activation)(x_1)

  x_2 = Conv1D(reduce_hidden * 2, kernel_size = conv_size2, padding='same' , kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(input_layer) 
  x_2 = BatchNormalization()(x_2)
  x_2 = Activation(cnn_activation)(x_2)
  x_2 = Conv1D(reduce_hidden, kernel_size = 1, padding='same' , kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(x_2) 
  x_2 = BatchNormalization()(x_2)
  x_2 = Activation(cnn_activation)(x_2)

  x_3 = Conv1D(reduce_hidden * 2, kernel_size = conv_size3, padding='same' , kernel_regularizer  = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(input_layer) 
  x_3 = BatchNormalization()(x_3)
  x_3 = Activation(cnn_activation)(x_3)
  x_3 = Conv1D(reduce_hidden, kernel_size = 1, padding='same' , kernel_regularizer = l1_l2(l1=l1_weight, l2=l2_weight), kernel_initializer=cnn_initializer, data_format = 'channels_last')(x_3) 
  x_3 = BatchNormalization()(x_3)
  x_3 = Activation(cnn_activation)(x_3)

  x = Add()([x_0, x_1, x_2, x_3])

  # Transformer block
  mha = LayerNormalization()(x)
  mha = MultiheadAttention(
      value_size = reduce_hidden // num_heads,
      key_size =  reduce_hidden // num_heads,
      num_heads = num_heads,
      attention_dropout_rate = transformer_att_dropout,
      relative_positions = transformer_rel_pos,
      relative_position_functions = transformer_pos_fncs,
      positional_dropout_rate = transformer_pos_dropout)(mha)
  mha = Add()([x, mha])
  lin = LayerNormalization()(mha)
  lin = snt.Linear(reduce_hidden*2)(lin)
  lin = Activation('swish')(lin)
  lin = snt.Linear(reduce_hidden)(lin)

  x = Add()([mha, lin])
  mha = LayerNormalization()(x)
  mha = MultiheadAttention(
      value_size = reduce_hidden // num_heads,
      key_size =  reduce_hidden // num_heads,
      num_heads = num_heads,
      attention_dropout_rate = transformer_att_dropout,
      relative_positions = transformer_rel_pos,
      relative_position_functions = transformer_pos_fncs,
      positional_dropout_rate = transformer_pos_dropout)(mha)
  mha = Add()([x, mha])
  lin = LayerNormalization()(mha)
  lin = snt.Linear(reduce_hidden*2)(lin)
  lin = Activation('swish')(lin)
  lin = snt.Linear(reduce_hidden)(lin)
  x = Add()([mha, lin])

  # LSTM
  x = Bidirectional(LSTM(lstm_units, 
                         return_sequences=False,
                         kernel_initializer = lstm_initializer,
                         activation = lstm_activation))(x)

  # Linear output layer
  output_layer = Dense(1, 
                       kernel_regularizer = l1_l2(l1=l1_weight, l2=l2_weight),
                       activation='linear', 
                       kernel_initializer=output_initializer, 
                       use_bias=True )(x) 
  
  model = Model(input_layer, output_layer)
  opt = tf.keras.optimizers.RMSprop(lr)
  return model, opt

In [46]:
# Compile model with the TPU strategy
with strategy.scope():
  model, opt = create_model()
  model.compile(optimizer=opt, loss=loss,metrics=[r_square, correlation_coefficient], 
                  steps_per_execution = 500)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(128, 110, 4)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (128, 110, 192)      11712       ['input_1[0][0]']                
                                                                                                  
 conv1d_2 (Conv1D)              (128, 110, 192)      15552       ['input_1[0][0]']                
                                                                                                  
 conv1d_4 (Conv1D)              (128, 110, 192)      19392       ['input_1[0][0]']                
                                                                                              

### Model fitting

In [47]:
model.fit(train_dataset.repeat().batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE), 
          validation_data = validation_dataset.repeat().batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE),
          steps_per_epoch = N_train_batches, validation_steps = N_val_batches,
          epochs=epochs , callbacks=callbacks_list) 
 

Epoch 1/10
 2500/36743 [=>............................] - ETA: 11:23 - loss: 2.4339 - r_square: 0.2991 - correlation_coefficient: 0.3204WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 2.9998s vs `on_train_batch_end` time: 6.9826s). Check your callbacks.


36743/36743 [==============================] - ETA: 0s - loss: 0.9097 - r_square: 0.4707 - correlation_coefficient: 0.4870
Epoch 1: val_correlation_coefficient improved from -inf to 0.54164, saving model to /content/drive/MyDrive/Colab Notebooks/DREAM/Final/Final_model.h5
36743/36743 [==============================] - 484s 13ms/step - loss: 0.9097 - r_square: 0.4707 - correlation_coefficient: 0.4870 - val_loss: 0.5036 - val_r_square: 0.5338 - val_correlation_coefficient: 0.5416
Epoch 2/10
36743/36743 [==============================] - ETA: 0s - loss: 0.4822 - r_square: 0.5395 - correlation_coefficient: 0.5523
Epoch 2: val_correlation_coefficient improved from 0.54164 to 0.55608, saving model to /content/drive/MyDrive/Colab Notebooks/DREAM/Final/Final_model.h5
36743/36743 [==============================] - 459s 12ms/step - loss: 0.4822 - r_square: 0.5395 - correlation_coefficient: 0.5523 - val_loss: 0.4672 - val_r_square: 0.5463 - val_correlation_coefficient: 0.5561
Epoch 3/10
36743/367

In [50]:
# Load weights of the best performing model
model.load_weights(wd + 'Final_model.h5')

# Re-evaluate the model
loss, r2, corr = model.evaluate( validation_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE), 
               verbose=2)
print("Model pearson correlation: {:5.2f}".format(corr))

93/93 - 7s - loss: 0.4488 - r_square: 0.5612 - correlation_coefficient: 0.5692 - 7s/epoch - 78ms/step
Model pearson correlation:  0.57


## Predictions

In [61]:
test_df = pd.read_csv(wd + 'test_sequences.txt', sep = '\t', header = None)

In [55]:
def old_seq2feature(data):
    A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
    C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
    G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
    T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
    N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

    mapper = {'A':A_onehot,'C':C_onehot,'G':G_onehot,'T':T_onehot,'N':N_onehot}

    for i in (range(0,len(data))) : 
        if (len(data[i]) > 110) :
            data[i] = data[i][-110:]
        elif (len(data[i]) < 110) : 
            while (len(data[i]) < 110) :
                data[i] = 'N'+data[i]

    transformed = np.asarray(([[mapper[k] for k in (data[i])] for i in (range(len(data)))]))
    return transformed


with open(os.path.join(wd + 'test_sequences.txt')) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)

    

sequences = [di[0] for di in d]
test_sequences= old_seq2feature(sequences)
print(test_sequences.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_

(71103, 110, 4)


In [58]:
X = test_sequences
batch_size = 1024
n_padding = (batch_size*(X.shape[0]//batch_size + 1) - X.shape[0])
X_padded = np.concatenate((X,np.repeat(X[0:1,:,:],n_padding,axis=0)))
Y_pred_padded = model.predict(X_padded , batch_size = batch_size , verbose=1)
Y_pred = Y_pred_padded[:X.shape[0]]

70/70 [==============================] - 6s 91ms/step


In [63]:
test_df[1] = Y_pred

In [71]:
test_df.to_csv(wd + 'Predictions.tsv', sep = '\t', header = None, index=False)